In [20]:
import pandas as pd
import numpy as np

# Importing Dataset

In [21]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/fashion-mnist_train.csv")

# Splitting and Scalling

In [22]:
from sklearn.model_selection import train_test_split as split

train_x, test_x, train_y, test_y = split(df.drop("label", axis=1), df["label"], test_size=0.2, random_state=42)

# Scalling
train_x = train_x/255.0
test_x = test_x/255.0

# GPU availability

In [23]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

# Custom Dataset Class

In [24]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features.values, dtype=torch.float32).reshape(-1, 1, 28, 28)
    self.labels = torch.tensor(labels.values, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]

In [25]:
# Object of Custom Dataset
train_dataset = CustomDataset(train_x, train_y)
test_dataset = CustomDataset(test_x, test_y)

# Object of custom DataLoader class
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Deep Learning Model

In [26]:
import torch.nn as nn

class MyNN(nn.Module):

  def __init__(self, input_features):
    super().__init__()

    self.feature = nn.Sequential(
        nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(32, 64, kernel_size=3, padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7, 128),
        nn.ReLU(),
        nn.Dropout(0.5),

        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Dropout(0.5),

        nn.Linear(64, 10)
    )

  def forward(self, x):
    x = self.feature(x)
    x = self.classifier(x)
    return x

# Model define and Important Parameter

In [27]:
learning_rate = 0.001
epochs = 100

model = MyNN(1)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Model Timeline

In [28]:
for epoch in range(epochs):
  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:
    # move the model to GPU
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    # forward pass
    output = model(batch_features)
    loss = criterion(output, batch_labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_epoch_loss += loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_epoch_loss/len(train_loader)}")

Epoch: 1, Loss: 0.5863301141659418
Epoch: 2, Loss: 0.38970447597404323
Epoch: 3, Loss: 0.34508937140305834
Epoch: 4, Loss: 0.3175976548691591
Epoch: 5, Loss: 0.29732017608359457
Epoch: 6, Loss: 0.27670126940806705
Epoch: 7, Loss: 0.26163435582692424
Epoch: 8, Loss: 0.2529649303033948
Epoch: 9, Loss: 0.24189080001662175
Epoch: 10, Loss: 0.2313888669324418
Epoch: 11, Loss: 0.22450893944812317
Epoch: 12, Loss: 0.21705436176309983
Epoch: 13, Loss: 0.21302719017739097
Epoch: 14, Loss: 0.20764232809717456
Epoch: 15, Loss: 0.19993171472164492
Epoch: 16, Loss: 0.19300458807249865
Epoch: 17, Loss: 0.18681095873750747
Epoch: 18, Loss: 0.18755717403565844
Epoch: 19, Loss: 0.1823848954460894
Epoch: 20, Loss: 0.17980982643024376
Epoch: 21, Loss: 0.17154527723851304
Epoch: 22, Loss: 0.17374505145506314
Epoch: 23, Loss: 0.16940087427943945
Epoch: 24, Loss: 0.16491933066832523
Epoch: 25, Loss: 0.1651654768618755
Epoch: 26, Loss: 0.16138617706221217
Epoch: 27, Loss: 0.15860941307557125
Epoch: 28, Loss:

# Model evaluation

In [31]:
model.eval()

total=0
correct=0
with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9151666666666667


In [32]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9799583333333334
